<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S56_diffdrive_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U gtbook

In [87]:
# No imports for now

# Deep Learning

> Stochastic gradient descent and the like.

**This Section is still in draft mode and was released for adventurous spirits (and TAs) only.**

In [1]:
from gtbook.display import randomImages
from IPython.display import display
display(randomImages(5, 6, "steampunk", 1))

## Supervised Learning Setup

> From data, learn concept.

In the **supervised learning** setup, we have a large number of examples of inputs $x$ and corresponding labels $y$.
We will often refer to the *training dataset* as $D$, consisting of pairs $(x,y)$.
If the labels $y$ are *discrete*, we talk about a supervised **classification** problem. If the labels $y$ are *continuous*, this is called a supervised **regression** problem.

## Gradient Descent

> Calculate gradient, reduce loss.

A neural network output, and in particular a CNN, depends on the large set of continuous weights $W$ that make up its convolutional layers, pooling layers, and fully connected layers. When we train a neural networks, we adjust its weights $W$ to perform better on the task at hand, be it classification or regression. To measure whether the model performs "better", we find a **loss function**. To adjust the weights, we calculate the gradient of the loss function with respect to each of the weights, and adjust the weights accordingly. That procedure is called **gradient descent**.

## Stochastic Gradient Descent

> Calculate approximate gradient, probably reduce loss.

Stochastic gradient descent is an approximate gradient descent procedure, to cope with the very large data sets typically thrown at supervised problems. It is typically impossible to calculate the *exact* gradient, which requires looping over all the examples, which can run in the millions (or billions). An easy approximation scheme is to *randomly sample* a small subset of the examples, and calculate the gradient of the weights using only those examples. The upside is that this is much faster, but the downside is that this is only approximate. Hence, if we adjust weights with this approximate gradient, we might or might not make progress on the task. This procedure is called **stochastic gradient descent**, and it works amazingly well in practice.

## Loss Functions

> A loss function for every occasion.

Different tasks require different loss functions, and a lot of creativity and research goes into crafting loss functions for complex tasks. For "vanilla" regression tasks, we typically use a **sum of squares differences** loss function as we already encountered before:

$$L_{\text{SSD}}(W; D) \doteq \sum_{(x,y)\in D}|f(x;W)-y|^2$$

Above $f(x;W)$ is the continuous prediction function implemented by the neural network, where $W$ represents the weights in all layers. Note that the formula above can be easily generalized to vector-valued labels $y$.

For classification, the **cross entropy** loss function is very popular: it measures the average disagreement of the predicted labels with the ground truth labels:

$$L_{\text{CE}}(W; D) \doteq \sum_c \sum_{(x,y=c)\in D}\log\frac{1}{p_c(x;W)}$$

This formula seems perhaps unintuitive and rather complicated. However, it is actually quite intuitive once you understand a few concepts.
In particular, in the multi-class classification problem we assume that the neural network outputs a probability $p_c(x;W)$ for every class $c\in[N]$, where $N$ is the number of classes. The quantity 

$$\log\frac{1}{p_c(x;W)}$$

is called the *surprise* that we should experience when seeing a label $y=c$. 
Indeed, for example, if we see class $3$ and the probability output by the network is $p_3(x;W)=1$, we are not surprised at all, as $\log\frac{1}{1}=\log 1 = 0$. 
However, if the probability is only $0.01$, our surprise is $\log\frac{1}{0.01}=\log 100 = 2$.
The lower the probability, the higher the surprise. Hence, the cross-entropy above measures the *average surprise* for seeing the labeled examples in the training data. After training, the network is the least surprised possible, hopefully, which is why it is an intuitive lost function to minimize.